# 0. Setup Paths

In [6]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [2]:
labels = [
    {'name':'Hello', 'id':1},
    {'name':'Yes', 'id':2},
    {'name':'No', 'id':3},
    {'name':'Thank You', 'id':4},
    {'name':'I Love You', 'id':5}
]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Covert xml label to csv

In [3]:
!python {SCRIPTS_PATH + '/generate_csv.py'} {'xml'} {IMAGE_PATH + '/train'} {IMAGE_PATH + '/train/output.csv'}
!python {SCRIPTS_PATH + '/generate_csv.py'} {'xml'} {IMAGE_PATH + '/test'} {IMAGE_PATH + '/test/output.csv'}

100%|█████████████████████████████████████████| 19/19 [00:00<00:00, 1992.44it/s]


# 3. Create TF records

In [4]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} {IMAGE_PATH + '/train/output.csv'} {ANNOTATION_PATH + '/label_map.pbtxt'} {IMAGE_PATH + '/train'} {"train.record"}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} {IMAGE_PATH + '/test/output.csv'} {ANNOTATION_PATH + '/label_map.pbtxt'} {IMAGE_PATH + '/test'} {"test.record"}


groups: 100%|██████████████████████████████████| 53/53 [00:00<00:00, 493.09it/s]
Successfully created the TFRecords: /Users/raginidwivedi/Documents/repos/RealTimeObjectDetection/train.record
groups: 100%|██████████████████████████████████| 19/19 [00:00<00:00, 624.48it/s]
Successfully created the TFRecords: /Users/raginidwivedi/Documents/repos/RealTimeObjectDetection/test.record


# 4. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [5]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 53687 (delta 16), reused 18 (delta 1), pack-reused 53654
Receiving objects: 100% (53687/53687), 569.66 MiB | 12.95 MiB/s, done.
Resolving deltas: 100% (36881/36881), done.


# 5. Copy Model Config to Training Folder

In [3]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [7]:
!mkdir {'Tensorflow/workspace/models/'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

# 6. Update Config For Transfer Learning

In [4]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [7]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [8]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [9]:
config

{'model': ssd {
   num_classes: 5
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [11]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [12]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

# 7. Train the model

In [15]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 8. Load Train Model From Checkpoint

In [13]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [14]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 9. Detect in Real-Time

In [15]:
import cv2 
import numpy as np

In [16]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [17]:
#cap.release()

In [18]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    print(image_np_with_detections)
    print("\n")
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

[[[30 42 54]
  [30 42 54]
  [30 42 54]
  ...
  [48 51 69]
  [48 51 69]
  [47 50 68]]

 [[30 42 54]
  [30 42 54]
  [30 42 54]
  ...
  [48 51 69]
  [48 51 69]
  [47 50 68]]

 [[27 41 53]
  [27 41 53]
  [27 41 53]
  ...
  [47 53 70]
  [47 53 70]
  [46 51 69]]

 ...

 [[43 32 38]
  [44 33 39]
  [45 34 40]
  ...
  [33 30 33]
  [30 27 30]
  [29 26 28]]

 [[42 31 37]
  [43 32 38]
  [44 33 39]
  ...
  [33 28 31]
  [32 26 30]
  [31 25 28]]

 [[42 31 37]
  [43 32 38]
  [43 32 38]
  ...
  [32 26 30]
  [32 26 30]
  [32 26 30]]]


[[[ 28  45  56]
  [ 27  44  55]
  [ 26  43  54]
  ...
  [116 134 160]
  [114 133 159]
  [114 133 159]]

 [[ 28  45  56]
  [ 27  44  55]
  [ 27  44  55]
  ...
  [117 135 162]
  [116 134 160]
  [116 134 160]]

 [[ 30  42  54]
  [ 30  42  54]
  [ 31  43  55]
  ...
  [114 137 163]
  [113 135 163]
  [113 135 163]]

 ...

 [[ 39  24  27]
  [ 39  24  27]
  [ 39  25  25]
  ...
  [161 181 194]
  [161 181 194]
  [163 184 196]]

 [[ 39  24  27]
  [ 39  24  27]
  [ 39  24  27]
  ...


[[[ 36  40  48]
  [ 34  38  46]
  [ 32  36  45]
  ...
  [103 129 152]
  [101 128 151]
  [100 127 150]]

 [[ 37  41  49]
  [ 35  39  47]
  [ 32  36  45]
  ...
  [104 130 153]
  [103 129 152]
  [100 127 150]]

 [[ 39  41  49]
  [ 37  38  47]
  [ 32  36  45]
  ...
  [102 130 153]
  [100 129 152]
  [ 98 127 150]]

 ...

 [[ 25  30  29]
  [ 26  31  30]
  [ 26  31  30]
  ...
  [136 160 176]
  [140 162 179]
  [143 165 182]]

 [[ 27  33  30]
  [ 30  35  32]
  [ 30  35  32]
  ...
  [132 159 175]
  [134 161 175]
  [137 164 178]]

 [[ 30  35  32]
  [ 32  38  35]
  [ 32  38  35]
  ...
  [130 157 173]
  [129 156 170]
  [132 160 173]]]


[[[ 18  35  46]
  [ 19  36  47]
  [ 18  35  46]
  ...
  [103 132 154]
  [102 130 153]
  [100 129 152]]

 [[ 19  36  47]
  [ 20  37  48]
  [ 19  36  47]
  ...
  [103 132 154]
  [102 130 153]
  [100 129 152]]

 [[ 20  38  46]
  [ 21  39  48]
  [ 20  38  46]
  ...
  [100 130 150]
  [100 130 150]
  [100 130 150]]

 ...

 [[ 31  31  33]
  [ 29  28  31]
  [ 28  27  30]
  

[[[ 41  53  60]
  [ 43  54  61]
  [ 43  55  60]
  ...
  [103 139 158]
  [103 139 158]
  [103 139 158]]

 [[ 41  53  60]
  [ 43  54  61]
  [ 44  56  61]
  ...
  [103 139 158]
  [103 139 158]
  [103 139 158]]

 [[ 41  52  62]
  [ 43  53  63]
  [ 44  54  64]
  ...
  [104 141 158]
  [104 141 160]
  [103 139 158]]

 ...

 [[226 129 113]
  [224 127 110]
  [224 125 109]
  ...
  [161 181 191]
  [161 181 191]
  [161 181 191]]

 [[234 128 104]
  [232 127 103]
  [232 124 101]
  ...
  [160 180 190]
  [160 180 190]
  [161 181 191]]

 [[230 124 101]
  [228 122  98]
  [229 120  97]
  ...
  [159 178 189]
  [160 180 190]
  [160 180 190]]]


[[[ 43  51  62]
  [ 45  53  63]
  [ 46  54  64]
  ...
  [109 137 162]
  [109 137 162]
  [109 137 162]]

 [[ 43  51  62]
  [ 45  53  63]
  [ 47  55  65]
  ...
  [109 137 162]
  [107 135 160]
  [107 135 160]]

 [[ 43  51  63]
  [ 45  52  64]
  [ 46  53  65]
  ...
  [109 137 162]
  [107 135 160]
  [106 134 159]]

 ...

 [[233 114  77]
  [233 114  77]
  [236 116  79]
  

[[[ 50  65  74]
  [ 51  67  76]
  [ 53  66  76]
  ...
  [119 161 184]
  [119 161 184]
  [119 161 184]]

 [[ 50  65  74]
  [ 51  67  76]
  [ 53  66  76]
  ...
  [120 162 185]
  [120 162 185]
  [120 162 185]]

 [[ 49  64  73]
  [ 50  65  74]
  [ 51  67  76]
  ...
  [121 164 185]
  [121 164 185]
  [121 164 185]]

 ...

 [[ 84  50  37]
  [ 83  49  36]
  [ 81  46  34]
  ...
  [174 200 203]
  [171 199 202]
  [168 197 199]]

 [[ 94  48  37]
  [ 96  49  38]
  [ 96  49  38]
  ...
  [163 193 197]
  [162 192 196]
  [160 191 195]]

 [[104  57  47]
  [104  57  47]
  [104  57  47]
  ...
  [152 183 187]
  [152 183 187]
  [151 182 186]]]


[[[ 47  60  76]
  [ 47  60  76]
  [ 48  61  77]
  ...
  [114 156 179]
  [115 158 179]
  [116 160 180]]

 [[ 48  61  77]
  [ 48  61  77]
  [ 48  61  77]
  ...
  [114 156 179]
  [115 158 179]
  [116 160 180]]

 [[ 48  61  77]
  [ 48  61  77]
  [ 46  61  77]
  ...
  [121 156 180]
  [122 158 181]
  [124 159 183]]

 ...

 [[ 92  55  45]
  [ 90  52  42]
  [ 90  52  42]
  

[[[ 43  70  84]
  [ 42  69  83]
  [ 40  68  81]
  ...
  [126 158 180]
  [125 156 179]
  [124 155 178]]

 [[ 42  69  83]
  [ 40  68  81]
  [ 39  67  80]
  ...
  [126 158 180]
  [125 156 179]
  [123 154 176]]

 [[ 43  67  79]
  [ 42  65  78]
  [ 41  64  76]
  ...
  [126 160 179]
  [123 157 179]
  [121 155 176]]

 ...

 [[100  48  40]
  [ 97  45  38]
  [ 96  44  37]
  ...
  [112 133 145]
  [123 142 153]
  [137 156 167]]

 [[125  52  38]
  [123  50  37]
  [122  52  38]
  ...
  [ 95 115 130]
  [104 124 139]
  [115 135 150]]

 [[137  64  51]
  [138  66  52]
  [138  67  53]
  ...
  [ 85 105 120]
  [ 88 108 123]
  [ 93 113 128]]]


[[[ 53  65  81]
  [ 51  64  80]
  [ 50  63  79]
  ...
  [133 155 179]
  [133 155 179]
  [132 154 178]]

 [[ 51  64  80]
  [ 50  63  79]
  [ 49  62  78]
  ...
  [134 156 180]
  [133 155 179]
  [131 153 176]]

 [[ 48  64  78]
  [ 47  63  76]
  [ 46  62  75]
  ...
  [129 156 177]
  [128 155 178]
  [127 153 176]]

 ...

 [[ 96  40  29]
  [ 97  41  30]
  [ 95  42  30]
  

...

[[[ 53  80  87]
  [ 52  79  86]
  [ 52  78  88]
  ...
  [159 205 231]
  [157 201 230]
  [157 201 230]]

 [[ 53  80  87]
  [ 52  79  86]
  [ 52  78  88]
  ...
  [158 203 230]
  [157 201 230]
  [157 201 230]]

 [[ 55  79  89]
  [ 55  79  89]
  [ 54  78  88]
  ...
  [158 203 232]
  [158 203 232]
  [157 201 230]]

 ...

 [[ 47  43  50]
  [ 47  43  50]
  [ 49  44  51]
  ...
  [183 205 220]
  [182 205 217]
  [182 205 217]]

 [[ 48  40  51]
  [ 48  40  51]
  [ 51  43  51]
  ...
  [182 204 219]
  [182 205 217]
  [182 205 217]]

 [[ 48  40  51]
  [ 49  41  52]
  [ 51  43  51]
  ...
  [181 203 218]
  [181 204 216]
  [182 205 217]]]


[[[ 46  79  89]
  [ 46  79  89]
  [ 49  79  90]
  ...
  [156 206 232]
  [152 204 230]
  [150 203 229]]

 [[ 41  74  85]
  [ 41  74  85]
  [ 45  76  87]
  ...
  [157 208 234]
  [153 206 231]
  [152 204 230]]

 [[ 39  72  82]
  [ 38  70  81]
  [ 42  72  83]
  ...
  [158 206 232]
  [157 205 231]
  [156 204 230]]

 ...

 [[ 50  38  47]
  [ 50  38  47]
  [ 50  38  47]
  

[[[ 52  79  95]
  [ 53  80  96]
  [ 53  80  94]
  ...
  [164 201 232]
  [166 201 232]
  [166 201 232]]

 [[ 53  80  96]
  [ 54  81  97]
  [ 54  82  95]
  ...
  [163 200 230]
  [165 200 230]
  [165 200 230]]

 [[ 55  79  96]
  [ 56  80  97]
  [ 56  80  97]
  ...
  [161 201 230]
  [161 201 230]
  [161 201 230]]

 ...

 [[ 43  44  42]
  [ 43  44  42]
  [ 44  45  43]
  ...
  [176 200 217]
  [178 201 215]
  [178 201 215]]

 [[ 43  44  42]
  [ 42  43  40]
  [ 42  43  40]
  ...
  [180 201 218]
  [178 201 215]
  [178 201 215]]

 [[ 43  44  42]
  [ 40  40  38]
  [ 38  39  37]
  ...
  [178 200 217]
  [180 202 217]
  [180 202 217]]]


[[[ 50  76  92]
  [ 51  77  94]
  [ 51  77  94]
  ...
  [157 203 234]
  [157 203 234]
  [156 202 233]]

 [[ 51  77  94]
  [ 51  77  94]
  [ 51  77  94]
  ...
  [156 202 233]
  [155 201 232]
  [155 201 232]]

 [[ 51  77  94]
  [ 52  79  95]
  [ 51  77  94]
  ...
  [151 201 229]
  [151 201 229]
  [151 201 229]]

 ...

 [[ 42  42  42]
  [ 42  42  42]
  [ 42  41  44]
  

[[[ 51  73  90]
  [ 51  73  90]
  [ 53  74  91]
  ...
  [150 204 232]
  [152 204 232]
  [152 204 232]]

 [[ 54  75  92]
  [ 54  75  92]
  [ 55  77  94]
  ...
  [150 204 232]
  [150 203 230]
  [150 203 230]]

 [[ 59  77  92]
  [ 60  78  93]
  [ 60  78  93]
  ...
  [149 201 229]
  [149 201 229]
  [149 201 229]]

 ...

 [[ 45  38  44]
  [ 46  40  45]
  [ 47  41  46]
  ...
  [181 203 225]
  [181 204 223]
  [180 203 222]]

 [[ 49  41  45]
  [ 49  41  45]
  [ 50  42  46]
  ...
  [180 202 224]
  [182 202 224]
  [182 202 224]]

 [[ 51  43  47]
  [ 51  43  47]
  [ 53  45  48]
  ...
  [179 201 223]
  [181 201 223]
  [181 201 223]]]


[[[ 51  72  92]
  [ 53  73  93]
  [ 57  75  95]
  ...
  [154 201 225]
  [153 199 224]
  [153 199 224]]

 [[ 53  73  93]
  [ 54  75  94]
  [ 57  75  95]
  ...
  [153 199 224]
  [152 198 223]
  [153 199 224]]

 [[ 55  74  91]
  [ 57  76  94]
  [ 58  77  95]
  ...
  [150 199 223]
  [153 199 224]
  [154 201 225]]

 ...

 [[ 58  40  52]
  [ 56  39  51]
  [ 55  38  49]
  

[[[ 53  74  91]
  [ 54  75  92]
  [ 55  77  94]
  ...
  [147 195 228]
  [146 194 227]
  [146 194 227]]

 [[ 54  75  92]
  [ 54  75  92]
  [ 55  77  94]
  ...
  [146 194 227]
  [144 192 226]
  [146 194 227]]

 [[ 56  75  92]
  [ 56  75  92]
  [ 59  76  94]
  ...
  [143 196 223]
  [142 194 224]
  [143 195 225]]

 ...

 [[ 53  42  42]
  [ 51  40  39]
  [ 49  37  37]
  ...
  [185 203 225]
  [184 201 224]
  [184 201 224]]

 [[ 54  44  41]
  [ 52  42  39]
  [ 49  38  35]
  ...
  [183 203 225]
  [184 201 224]
  [184 201 224]]

 [[ 54  44  41]
  [ 52  42  39]
  [ 50  39  36]
  ...
  [183 203 225]
  [184 201 224]
  [184 201 224]]]


[[[ 64  77  93]
  [ 64  77  93]
  [ 63  76  92]
  ...
  [143 202 231]
  [141 202 231]
  [141 202 231]]

 [[ 65  78  94]
  [ 65  78  94]
  [ 64  77  93]
  ...
  [144 203 232]
  [141 202 231]
  [141 202 231]]

 [[ 69  78  97]
  [ 69  78  97]
  [ 67  79  97]
  ...
  [144 203 232]
  [141 202 231]
  [141 202 231]]

 ...

 [[ 42  38  41]
  [ 42  38  41]
  [ 43  40  42]
  

[[[ 58  75  84]
  [ 58  75  84]
  [ 60  78  86]
  ...
  [157 202 229]
  [153 201 229]
  [152 199 228]]

 [[ 56  74  83]
  [ 58  75  84]
  [ 59  76  85]
  ...
  [157 202 229]
  [153 201 229]
  [152 199 228]]

 [[ 53  73  81]
  [ 54  74  83]
  [ 57  77  85]
  ...
  [153 201 228]
  [150 201 227]
  [150 201 227]]

 ...

 [[ 38  39  41]
  [ 40  42  44]
  [ 41  43  45]
  ...
  [185 207 228]
  [182 205 224]
  [181 204 223]]

 [[ 41  40  42]
  [ 42  41  44]
  [ 43  42  45]
  ...
  [185 206 226]
  [184 205 224]
  [183 204 223]]

 [[ 42  41  44]
  [ 42  41  44]
  [ 43  42  45]
  ...
  [184 205 224]
  [184 205 224]
  [183 204 223]]]


[[[ 67  74  94]
  [ 68  75  95]
  [ 66  77  93]
  ...
  [162 203 230]
  [162 203 230]
  [163 204 231]]

 [[ 64  71  90]
  [ 65  72  91]
  [ 65  75  92]
  ...
  [161 201 229]
  [162 203 230]
  [162 203 230]]

 [[ 59  69  88]
  [ 62  71  90]
  [ 64  73  92]
  ...
  [159 202 229]
  [161 201 230]
  [161 201 230]]

 ...

 [[ 42  45  44]
  [ 43  46  45]
  [ 43  45  47]
  

In [ ]:
#detections = detect_fn(input_tensor)